## 무비렌즈

In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

### 데이터 준비

In [74]:
# 데이터 로드

"""
필요한 컬럼은 유저, 아이템, rating

우선 간단하게 빨리 해보는게 중요하니,
rating이 5점이면 rating 컬럼을 1
아니라면 0로 바꾸자고.
"""


def getDataByScenario(scenario):
    """
    :param scenario: increase, fixed, user, item
    :return: dfs
    """
    dfs = []

    if scenario in ["increase", 'fixed']:
        for i in range(6):
            df = pd.read_csv(f"./dataset/Movielens/{scenario}/ml_1m_inc{i}.csv")
            dfs.append(df)

    if scenario in ["user", "item"]:
        for i in range(6):
            train = pd.read_csv(f"./dataset/Movielens/{scenario}/train_ml_1m_inc{i}.csv")
            test = pd.read_csv(f"./dataset/Movielens/{scenario}/test_ml_1m_inc{i}.csv")
            dfs.append((train, test))

    return dfs

## dataloader 정의

In [75]:
class MovielensDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user = self.df.iloc[idx]['user']
        item = self.df.iloc[idx]['item']
        rating = self.df.iloc[idx]['rating']
        return user, item, rating

## 모델 정의

In [76]:
# NCF 모델
class NCF(nn.Module):
    def __init__(self, n_users, n_movies, emb_size=8, hidden_size=64):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(n_users, emb_size)
        self.movie_embedding = nn.Embedding(n_movies, emb_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(emb_size * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, user_input, movie_input):
        user_embedded = self.user_embedding(user_input)
        movie_embedded = self.movie_embedding(movie_input)
        input_concat = torch.cat([user_embedded, movie_embedded], dim=-1)
        prediction = self.fc_layers(input_concat)
        return prediction

In [77]:
# gpu 설정
use_cuda = True

use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

## 모델 train/test 함수 정의

In [78]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    # print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [79]:
def recall_at_k(output, target, k):
    if len(output) < k:
        k = len(output)
    _, idx = torch.topk(output, k=k)
    hit = torch.sum(target[idx])
    return hit.float() / target.sum().float() if target.sum().float() else torch.Tensor([0])


def test(model, device, test_loader, k=20):
    model.eval()
    criterion = nn.BCELoss()

    test_loss = 0
    test_recall = 0
    with torch.no_grad():
        for user, item, rating in test_loader:
            user, item, rating = user.to(device), item.to(device), rating.to(device)
            output = model(user, item).squeeze()
            loss = criterion(output, rating.float())
            test_loss += loss.item()
            test_recall += recall_at_k(output, rating, k).item()  # recall@20 기준
    test_loss /= len(test_loader)
    test_recall /= len(test_loader)

    return test_loss, test_recall

### 모델 학습

1. Naive
2. EWC

1. Naive

우선 모든 데이터에 대해 incremental training을 하고 test해보자

In [80]:
### Config..
EPOCH = 1
SEED = 42
BATCH_SIZE = 64
N_USER = 6040
N_ITEM = 3952

In [81]:
def getNaiveResultByScenario(scenario):
    recall_list = []
    dfs = getDataByScenario(scenario)

    for i, df in enumerate(dfs):

        if i == 0:
            # base block train-test

            if scenario in ["increase", "fixed"]:
                train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=SEED)
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

            # 모델 객체 생성
            n_users = N_USER + 1
            n_movies = N_ITEM + 1
            model = NCF(n_users, n_movies).to(device)
            # 옵티마이저 설정
            optimizer = optim.Adam(model.parameters(), lr=0.001)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train(model, device, train_loader, optimizer, e)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* At {i} TASK recall20 = {recall20}\n")

        else:
            # inc block train-test

            # 데이터 준비
            if scenario in ["increase", "fixed"]:
                if i == len(dfs)-1:
                    break
                train_dataset = df
                test_dataset = dfs[i+1]
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train(model, device, train_loader, optimizer, e)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* At {i} TASK recall20 = {recall20}\n")

    avg_recall = sum(recall_list) / len(recall_list)
    print(f"{scenario} scenario avg recall : {avg_recall}")
    return avg_recall

In [82]:
naiveIncrease = getNaiveResultByScenario("increase")
naivefixed = getNaiveResultByScenario("fixed")
naiveUser = getNaiveResultByScenario("user")
naiveItem = getNaiveResultByScenario("item")

************** Train Start At TASK0


100%|██████████| 1/1 [01:46<00:00, 106.55s/it]


******* At 0 TASK recall20 = 0.5937409374526778

************** Train Start At TASK1


100%|██████████| 1/1 [00:18<00:00, 18.80s/it]


******* At 1 TASK recall20 = 0.5634372855623881

************** Train Start At TASK2


...

******* At 3 TASK recall20 = 0.6222622150182724

************** Train Start At TASK4


100%|██████████| 1/1 [00:07<00:00,  7.91s/it]


******* At 4 TASK recall20 = 0.6193636810162153

************** Train Start At TASK5


100%|██████████| 1/1 [00:08<00:00,  8.07s/it]


******* At 5 TASK recall20 = 0.6263254019281557

avg recall : 0.6136039475721912


2. EWC

In [83]:
# Task가 끝날 때 마다 optpar와 fisher를 저장해주는 함수.
def on_task_update(model, device, train_loader, optimizer, task_id, fisher_dict, optpar_dict):
    model.train()
    criterion = nn.BCELoss()
    optimizer.zero_grad()

    # accumulating gradients
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()

    fisher_dict[task_id] = {}
    optpar_dict[task_id] = {}

    # gradients accumulated can be used to calculate fisher
    for name, param in model.named_parameters():
        fisher_dict[task_id][name] = param.grad.data.clone().pow(2)  # 누적 grad 값
        optpar_dict[task_id][name] = param.data.clone()  # 최적 grad 값

In [84]:
# EWC를 적용한 train 함수
def train_ewc(model, device, train_loader, optimizer, epoch, task_id, fisher_dict, optpar_dict, ewc_lambda):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        train_loss += loss.item()

        # EWC 적용 부분
        for task in range(task_id):
            for name, param in model.named_parameters():
                fisher = fisher_dict[task][name]
                optpar = optpar_dict[task][name]
                train_loss += (fisher * (optpar - param).pow(2)).sum() * ewc_lambda

        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    # print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [85]:
def getEWCResultByScenario(scenario):
    recall_list = []
    dfs = getDataByScenario(scenario)
    # EWC에 필요한 변수
    fisher_dict = {}
    optpar_dict = {}
    ewc_lambda = 0.4  # ewc 강도 조절.. 높을수록 이전 파라미터의 중요도가 높아짐

    for i, df in enumerate(dfs):
        if i == 0:
            # base block train-test

            if scenario in ["increase", "fixed"]:
                train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=SEED)
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)



            # 모델 객체 생성
            n_users = N_USER + 1
            n_movies = N_ITEM + 1
            model = NCF(n_users, n_movies).to(device)
            # 옵티마이저 설정
            optimizer = optim.Adam(model.parameters(), lr=0.001)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train_ewc(model, device, train_loader, optimizer, e, i, fisher_dict, optpar_dict, ewc_lambda)
            on_task_update(model, device, train_loader, optimizer, i, fisher_dict, optpar_dict)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* At {i} TASK recall20 = {recall20}\n")

        else:
            # inc block train-test

            # 데이터 준비
            if scenario in ["increase", "fixed"]:
                if i == len(dfs)-1:
                    break
                train_dataset = df
                test_dataset = dfs[i+1]
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train_ewc(model, device, train_loader, optimizer, e, i, fisher_dict, optpar_dict, ewc_lambda)
            on_task_update(model, device, train_loader, optimizer, i, fisher_dict, optpar_dict)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* At {i} TASK recall20 = {recall20}\n")

    avg_recall = sum(recall_list) / len(recall_list)
    print(f"{scenario} scenario avg recall : {avg_recall}")
    return avg_recall

In [ ]:
ewcIncrease = getEWCResultByScenario("increase")
ewcfixed = getEWCResultByScenario("fixed")
ewcUser = getEWCResultByScenario("user")
ewcItem = getEWCResultByScenario("item")

************** Train Start At TASK0


100%|██████████| 1/1 [01:37<00:00, 97.79s/it]


******* At 0 TASK recall20 = 0.5911402050594691

************** Train Start At TASK1


100%|██████████| 1/1 [00:18<00:00, 18.29s/it]


******* At 1 TASK recall20 = 0.5559481129205351

************** Train Start At TASK2


...

100%|██████████| 1/1 [00:20<00:00, 20.39s/it]


******* At 3 TASK recall20 = 0.5494031014921572

************** Train Start At TASK4


100%|██████████| 1/1 [00:22<00:00, 22.72s/it]


******* At 4 TASK recall20 = 0.5664144688314968

avg recall : 0.5634479458929749
************** Train Start At TASK0


100%|██████████| 1/1 [01:38<00:00, 98.14s/it]


In [ ]:
print(f"""
naiveIncrease: {naiveIncrease}
naivefixed: {naivefixed}
naiveUser: {naiveUser}
naiveItem" {naiveItem}
""")

In [ ]:
print(f"""
ewcIncrease: {ewcIncrease}
ewcfixed: {ewcfixed}
ewcUser: {ewcUser}
ewcItem" {ewcItem}
""")

In [ ]:
print(f"""
{ewcIncrease-naiveIncrease}
{ewcfixed-naivefixed}
{ewcUser-naiveUser}
{ewcItem-naiveItem}
""")